In [ ]:
import config
import torch
import sys
sys.path.append('../util')
sys.path.append('../others_networks')

#Faça o clone do repositorio: #https://github.com/LeeJunHyun/Image_Segmentation/blob/master/network.py
#Adicione o local do clone para poder fazer os imports das UNets
sys.path.append("/mnt/TUDAO/Image_Segmentation")
from network import U_Net
from network import AttU_Net
from network import R2AttU_Net

from util import measure_inference_speed, clear_gpu
from thop import profile

from DatasetReader import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_path          = './LeeJunUNets/'

In [ ]:
in_channels = config.in_channels
num_classes = config.num_classes

train_loader, test_loader, val_loader = get_datasets(dataset_dir=config.dataset_path, 
                                                    resolution=config.dataset_resolution, 
                                                    batch_size=config.batch_size)

In [ ]:
def evaluate(model, model_filename=None):
    state_dict = torch.load(model_filename, map_location='cpu')
    model.load_state_dict(state_dict['model_state_dict'], strict=False)

    #CONTA O FPS    
    fps, time_per_image, cpu_fps, cpu_time_per_image = measure_inference_speed(model, val_loader, steps=100)
    print("GPU FPS:",fps,       "GPU Time per image:",time_per_image)
    print("CPU FPS:",cpu_fps,   "CPU Time per image:",cpu_time_per_image)

    #Conta os GFLOPS
    input = torch.randn(1, 3, 224, 224)
    flops, params = profile(model, inputs=(input,))
    flops_g = flops / 1e9
    params_m = params / 1e6
    print("Axial")
    print(f"GFLOPs: {flops_g:.3f}")
    print(f"Params (M): {params_m:.3f}")

    del model
    clear_gpu()
    

In [4]:
model = U_Net(img_ch=in_channels, output_ch=num_classes)
evaluate(model, model_filename=f"{result_path}/UNet-1.pth")

GPU FPS: 144.81966894582573 GPU Time per image: 6.905 ms
CPU FPS: 3.089389694039062 CPU Time per image: 323.689 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
Axial
GFLOPs: 50.166
Params (M): 34.527


In [5]:
model = AttU_Net(img_ch=in_channels, output_ch=num_classes)
evaluate(model, model_filename=f"{result_path}/AttU_Net-1.pth")

GPU FPS: 131.44017368180815 GPU Time per image: 7.608 ms
CPU FPS: 2.5159402660006847 CPU Time per image: 397.466 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
Axial
GFLOPs: 51.015
Params (M): 34.879


In [6]:
model = R2AttU_Net(img_ch=in_channels, output_ch=num_classes)
evaluate(model, model_filename=f"{result_path}/R2AttU_Net-1.pth")

GPU FPS: 53.360581218600586 GPU Time per image: 18.740 ms
CPU FPS: 1.0789186517707199 CPU Time per image: 926.854 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
Axial
GFLOPs: 117.928
Params (M): 39.443


In [ ]:
from ULite import ULite
model = ULite(in_channels=in_channels, out_channels=num_classes)
evaluate(model, model_filename=f"ULite/ULite-1-epochs300.pth")

GPU FPS: 939.944157043783 GPU Time per image: 1.064 ms
CPU FPS: 21.688481206480404 CPU Time per image: 46.107 ms
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
Axial
GFLOPs: 0.580
Params (M): 0.878


In [ ]:
from UNext import UNext
model = UNext(input_channels=in_channels, num_classes=num_classes)
evaluate(model, model_filename=f"UNext/UNext-1-epochs300.pth")

/home/alan/miniconda3/envs/pytorch5070/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alan/miniconda3/envs/pytorch5070/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


GPU FPS: 2895.7551075710758 GPU Time per image: 0.345 ms
CPU FPS: 79.6439278197092 CPU Time per image: 12.556 ms
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_softmax() for <class 'torch.nn.modules.activation.Softmax'>.
Axial
GFLOPs: 0.439
Params (M): 1.472


In [ ]:
from DeepLabV3.py import DeepLabV3MobilenetV2Wrapper
model = DeepLabV3MobilenetV2Wrapper(in_channels=in_channels, out_channels=num_classes)
evaluate(model, model_filename=f"DeepLabV3/DeepLabV3-1-epochs300.pth")

GPU FPS: 2484.107660553583 GPU Time per image: 0.403 ms
CPU FPS: 48.42180136335185 CPU Time per image: 20.652 ms
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU6'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
Axial
GFLOPs: 0.821
Params (M): 12.648


In [ ]:
from UMergeNet import UMergeNet, ConvType

In [ ]:
model = UMergeNet(in_channels=3, out_channels=1, conv_type=ConvType.Axial)
evaluate(model, model_filename=f"umergenet/UMergeNet-axial-1-epochs300.pth")

GPU FPS: 929.1563281103 GPU Time per image: 1.076 ms
CPU FPS: 24.476932394885015 CPU Time per image: 40.855 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
Axial
GFLOPs: 0.883
Params (M): 1.727


In [ ]:
model = UMergeNet(in_channels=3, out_channels=1, conv_type=ConvType.Axial, merger_groups='dw', encoder_groups='dw', decoder_groups='dw')
evaluate(model, model_filename=f"umergenet/UMergeNet-axial-dw-1-epochs300.pth")

GPU FPS: 1078.3677628945545 GPU Time per image: 0.927 ms
CPU FPS: 25.120243161781428 CPU Time per image: 39.809 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
Axial
GFLOPs: 0.481
Params (M): 0.685


In [ ]:
model = UMergeNet(in_channels=3, out_channels=1, conv_type=ConvType.Atrous)
evaluate(model, model_filename=f"umergenet/UMergeNet-atrous-1-epochs300.pth")

GPU FPS: 932.8770855598024 GPU Time per image: 1.072 ms
CPU FPS: 25.23646281524574 CPU Time per image: 39.625 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
Axial
GFLOPs: 1.187
Params (M): 2.487


In [ ]:
model = UMergeNet(in_channels=3, out_channels=1, conv_type=ConvType.Atrous, merger_groups='dw', encoder_groups='dw', decoder_groups='dw')
evaluate(model, model_filename=f"umergenet/UMergeNet-atrous-dw-1-epochs300.pth")

GPU FPS: 1307.747173953585 GPU Time per image: 0.765 ms
CPU FPS: 29.04769076580094 CPU Time per image: 34.426 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
Axial
GFLOPs: 0.513
Params (M): 0.702


In [ ]:
model = UMergeNet(in_channels=3, out_channels=1, conv_type=ConvType.Standard)
evaluate(model, model_filename=f"umergenet/UMergeNet-normal-1-epochs300.pth")

GPU FPS: 731.1839147711239 GPU Time per image: 1.368 ms
CPU FPS: 24.106542736368457 CPU Time per image: 41.483 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
Axial
GFLOPs: 1.852
Params (M): 4.150


In [ ]:
model = UMergeNet(in_channels=3, out_channels=1, conv_type=ConvType.Standard, merger_groups='dw', encoder_groups='dw', decoder_groups='dw')
evaluate(model, model_filename=f"umergenet/UMergeNet-normal-dw-1-epochs300.pth")

GPU FPS: 1168.8629562927379 GPU Time per image: 0.856 ms
CPU FPS: 28.76561214016652 CPU Time per image: 34.764 ms
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
Axial
GFLOPs: 0.583
Params (M): 0.740
